In [12]:
import pandas as pd
import numpy as np

In [13]:
train = pd.read_csv("../data/train.csv")
train = train.fillna("unknown")

In [14]:
train[:5]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [15]:
target_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
num_classes = len(target_labels)

In [16]:
test = pd.read_csv("../data/test.csv")
test = test.fillna("unknown")

In [17]:
test[:5]

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [18]:
train[train["toxic"] == 1][:5]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


In [19]:
train_y_pd = train[target_labels]
train_y = train_y_pd.values
train_y[0]

array([0, 0, 0, 0, 0, 0])

In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [21]:
max_fatures = 25000
max_len = 100

In [22]:
tokenizer = Tokenizer(num_words=max_fatures)
corpus = train["comment_text"].append(test["comment_text"])
tokenizer.fit_on_texts(list(corpus.values))

In [23]:
def calc_text_uniq_words(samples):
    unique_words = set()
    for sample in samples:
        for word in sample.split():
            unique_words.add(word)
    return len(unique_words)

calc_text_uniq_words(corpus.values)

962229

In [24]:
def calc_text_len_word(samples):
    lens = []
    for sample in samples:
        for word in sample.split():
            lens.append(len(word))
    return mean(lens)

In [25]:
train_x = tokenizer.texts_to_sequences(train["comment_text"])
train_x = pad_sequences(train_x, maxlen=max_len)

In [26]:
print(train_x[0])
train_x[0].shape

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0   733    78     1   140   131   182    30
   712  4438 10284  1252    86   368    51  2230 14039    49  6744    15
    60  2624   151     7  2832    33   115  1246 16129  2517     5    50
    59   256     1   370    31     1    46    29   144    72  3931    89
  4208  6368  2687  1183]


(100,)

In [27]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.models import Sequential

In [17]:
model = Sequential([
    Embedding(max_fatures,256, input_length=train_x.shape[1]),
    Bidirectional(LSTM(128, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dense(num_classes, activation="sigmoid")
])

In [18]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 256)          6400000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 256)          394240    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 774       
Total params: 6,827,910
Trainable params: 6,827,910
Non-trainable params: 0
_________________________________________________________________


In [20]:
print(train_x.shape)
print(train_y.shape)

(159571, 100)
(159571, 6)


In [33]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

stopping = EarlyStopping(patience=2)

checkpoint = ModelCheckpoint("weights.{epoch:03d}--{val_loss:.2f}.hdf5", save_best_only=False)

In [22]:
model.fit(train_x, train_y, validation_split=0.2, batch_size=256, epochs=5, callbacks=[stopping, checkpoint])

Train on 127656 samples, validate on 31915 samples
Epoch 1/5
127656/127656 [==============================] - 98s 764us/step - loss: 0.0940 - acc: 0.9724 - val_loss: 0.0518 - val_acc: 0.9816
Epoch 2/5
127656/127656 [==============================] - 96s 752us/step - loss: 0.0478 - acc: 0.9825 - val_loss: 0.0501 - val_acc: 0.9817
Epoch 3/5
127656/127656 [==============================] - 95s 746us/step - loss: 0.0418 - acc: 0.9841 - val_loss: 0.0530 - val_acc: 0.9822


In [23]:
model.save("bi_lstm_1.h5")

In [32]:
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.layers import GRU

In [72]:
model = Sequential([
    Embedding(max_fatures,256, input_length=train_x.shape[1]),
    Conv1D(32, kernel_size=5, padding="same", activation="relu"),
    MaxPooling1D(pool_size=3),
    Dropout(0.3),
    Conv1D(64, kernel_size=5, padding="same", activation="relu"),
    MaxPooling1D(pool_size=3),
    Dropout(0.4),
    Conv1D(128, kernel_size=5, padding="same", activation="relu"),
    MaxPooling1D(pool_size=3),
    Dropout(0.3),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.4),
    Dense(6, activation="sigmoid")
])

In [73]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [74]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 100, 128)          3200000   
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 100, 32)           20512     
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 33, 32)            0         
_________________________________________________________________
dropout_34 (Dropout)         (None, 33, 32)            0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 33, 64)            10304     
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 11, 64)            0         
_________________________________________________________________
dropout_35 (Dropout)         (None, 11, 64)            0         
__________

In [75]:
model.fit(train_x, train_y, validation_split=0.2, batch_size=512, epochs=5, callbacks=[stopping, checkpoint])

Train on 127656 samples, validate on 31915 samples
Epoch 1/5
127656/127656 [==============================] - 10s 76us/step - loss: 0.1500 - acc: 0.9593 - val_loss: 0.0668 - val_acc: 0.9779
Epoch 2/5
127656/127656 [==============================] - 7s 55us/step - loss: 0.0565 - acc: 0.9802 - val_loss: 0.0523 - val_acc: 0.9816
Epoch 3/5
127656/127656 [==============================] - 7s 55us/step - loss: 0.0465 - acc: 0.9828 - val_loss: 0.0536 - val_acc: 0.9816


In [63]:
model.save('conv_bi_lstm.h5')

In [7]:
def load_vectors(path):
    with open(path) as lines:
        w2v = {line.split()[0]: np.array( list( map(float,line.split()[1:]) ))
           for line in lines}
        embedding_matrix = np.zeros((len(w2v) + 1, 100))
        for i,word in enumerate(w2v):
            embedding_vector = w2v[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
        return w2v, embedding_matrix
    

In [8]:
w2v, embedding_matrix = load_vectors("../data/word2vec_gensim.txt")

In [9]:
len(w2v)

119559

In [10]:
embedding_matrix[0]

array([ 0.009009, -0.092303,  0.046214, -0.000558, -0.20167 ,  0.005846,
       -0.101494, -0.124898,  0.094279,  0.042075,  0.11357 ,  0.089216,
       -0.14359 , -0.022416, -0.03374 , -0.151293,  0.006691,  0.018786,
       -0.018465,  0.121886,  0.068761, -0.002243,  0.061386, -0.289279,
        0.039515,  0.184506,  0.252   , -0.07177 ,  0.12287 , -0.251433,
        0.019958,  0.015398, -0.386273,  0.018585,  0.087083,  0.030035,
        0.239704,  0.101596,  0.026426,  0.235025,  0.093816, -0.097863,
        0.058541,  0.123449,  0.070184, -0.069415, -0.133104,  0.096352,
       -0.055501,  0.134075,  0.130833, -0.135855, -0.067685, -0.081129,
        0.026798, -0.070918, -0.006838, -0.102813,  0.099745,  0.045488,
       -0.050391,  0.252638, -0.130138, -0.08353 ,  0.140009,  0.106841,
        0.039647, -0.130847,  0.139748, -0.135322, -0.047111, -0.068008,
       -0.035898,  0.114875, -0.101487,  0.109524, -0.011924,  0.030436,
        0.059811,  0.021645, -0.176884,  0.075816, 

In [28]:
model = Sequential([
    Embedding(len(w2v) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False),
    Bidirectional(LSTM(128, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dense(num_classes, activation="sigmoid")
])

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          11956000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 256)          234496    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 774       
Total params: 12,224,166
Trainable params: 268,166
Non-trainable params: 11,956,000
__________________________________________________________

In [30]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

In [36]:
checkpoint = ModelCheckpoint("./models/glove_my_vectors_bi_lstm.h5", save_best_only=True)

In [50]:
model.fit(train_x, train_y, validation_split=0.2, batch_size=256, epochs=2, callbacks=[checkpoint])

Train on 127656 samples, validate on 31915 samples
Epoch 1/2
127656/127656 [==============================] - 85s 665us/step - loss: 0.0692 - acc: 0.9768 - val_loss: 0.0742 - val_acc: 0.9758
Epoch 2/2
127656/127656 [==============================] - 84s 661us/step - loss: 0.0677 - acc: 0.9772 - val_loss: 0.0747 - val_acc: 0.9757


In [43]:
test_tokenized = tokenizer.texts_to_sequences(test["comment_text"])

In [44]:
from keras.models import load_model

In [45]:
MODEL = "./models/glove_my_vectors_bi_lstm.h5"

In [46]:
model = load_model(MODEL)

In [47]:
test_tokenized = pad_sequences(test_tokenized, maxlen=max_len)

In [48]:
predicted  = model.predict(test_tokenized)

In [49]:
predicted[0]

array([0.94439274, 0.1186297 , 0.7227583 , 0.12468548, 0.7285815 ,
       0.32733577], dtype=float32)

In [83]:
predicted[:5]

array([[  9.94870186e-01,   1.50896505e-01,   9.59150553e-01,
          3.39063890e-02,   8.79422963e-01,   1.12759568e-01],
       [  6.44919928e-03,   1.53828933e-05,   2.07967474e-03,
          3.35066288e-05,   1.96722965e-03,   2.60747183e-04],
       [  3.01423436e-03,   1.62482102e-05,   6.29154791e-04,
          4.17441297e-05,   8.37880711e-04,   2.21316121e-04],
       [  1.22966466e-03,   1.37287270e-06,   1.58192270e-04,
          1.02902795e-05,   2.29060737e-04,   2.73610967e-05],
       [  3.12186200e-02,   2.47693213e-04,   7.47623108e-03,
          3.31957068e-04,   7.70709990e-03,   1.06239645e-03]], dtype=float32)

In [84]:
submission = pd.DataFrame(data=predicted,columns=target_labels,index=test["id"] )
submission[:5]

,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
00001cee341fdb12,0.994870,0.150897,0.959151,0.033906,0.879423,0.112760
0000247867823ef7,0.006449,0.000015,0.002080,0.000034,0.001967,0.000261
00013b17ad220c46,0.003014,0.000016,0.000629,0.000042,0.000838,0.000221
00017563c3f7919a,0.001230,0.000001,0.000158,0.000010,0.000229,0.000027
00017695ad8997eb,0.031219,0.000248,0.007476,0.000332,0.007707,0.001062


In [85]:
submission.to_csv("./submission_glove_bi_lstm.csv")